<a href="https://colab.research.google.com/github/fayzafz/IF_B_2021/blob/main/audio_dwt_compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install streamlit numpy PyWavelets


In [11]:
# -*- coding: utf-8 -*-
"""
Audio Compression with DWT and DCT
"""

import streamlit as st
import numpy as np
import pywt
import wave
import struct
from io import BytesIO
from scipy.fftpack import dct, idct

# Fungsi untuk membaca file audio
def read_audio(file_bytes):
    with wave.open(BytesIO(file_bytes), 'rb') as wav_file:
        params = wav_file.getparams()
        frames = wav_file.readframes(params.nframes)
        audio = struct.unpack_from("%dh" % params.nframes, frames)
        audio = np.array(audio, dtype=np.int16)
    return audio, params

# Fungsi untuk menulis file audio
def write_audio(audio, params):
    with BytesIO() as buf:
        with wave.open(buf, 'wb') as wav_file:
            wav_file.setparams(params)
            frames = struct.pack("%dh" % len(audio), *audio)
            wav_file.writeframes(frames)
        return buf.getvalue()

# Fungsi untuk kompresi audio menggunakan DWT
def dwt_compress(audio, wavelet='db1', level=1):
    coeffs = pywt.wavedec(audio, wavelet, level=level)
    threshold = np.median(np.abs(coeffs[-level])) / 0.6745
    coeffs = list(map(lambda x: pywt.threshold(x, threshold, mode='soft'), coeffs))
    return coeffs

# Fungsi untuk dekompresi audio menggunakan DWT
def dwt_decompress(coeffs, wavelet='db1'):
    audio_reconstructed = pywt.waverec(coeffs, wavelet)
    audio_reconstructed = np.clip(audio_reconstructed, -32768, 32767)
    audio_reconstructed = np.array(audio_reconstructed, dtype=np.int16)
    return audio_reconstructed

# Fungsi untuk kompresi audio menggunakan DCT
def dct_compress(audio, quality=50):
    audio = audio - np.mean(audio)
    dct_audio = dct(audio, norm='ortho')
    threshold = np.percentile(np.abs(dct_audio), quality)
    dct_audio[np.abs(dct_audio) < threshold] = 0
    compressed_audio = idct(dct_audio, norm='ortho')
    compressed_audio = np.clip(compressed_audio, -32768, 32767)
    return np.array(compressed_audio, dtype=np.int16)

# Fungsi utama aplikasi Streamlit
def main():
    st.title('Kompresi Gambar dan Audio')
    st.write("Unggah file audio (WAV) dan kompres dengan Discrete Wavelet Transform (DWT) atau Discrete Cosine Transform (DCT).")

    # Navigasi sidebar
    with st.sidebar:
        selected = st.selectbox('Kompresi Gambar & Audio',
                                ['Beranda', 'Kompresi Audio'])

    if selected == 'Beranda':
        st.title('Selamat Datang di Kompresi Gambar dan Audio!')

    # Halaman kompresi audio
    if selected == 'Kompresi Audio':
        st.title('Kompresi Audio')
        st.write("Unggah file audio (WAV) dan kompres dengan Discrete Wavelet Transform (DWT) atau Discrete Cosine Transform (DCT).")

        uploaded_file = st.file_uploader("Pilih file audio", type=["wav"], accept_multiple_files=False)

        if uploaded_file is not None:
            st.write('File yang diunggah:', uploaded_file.name)

            audio, params = read_audio(uploaded_file.getvalue())

            algorithm = st.selectbox("Pilih Algoritma Kompresi:", ["DWT", "DCT"])
            if algorithm == "DWT":
                wavelet = st.selectbox("Pilih Wavelet:", ['db1', 'haar', 'sym2', 'coif1'])
                level = st.slider("Pilih Level DWT:", 1, 5, 1)
            elif algorithm == "DCT":
                quality = st.slider("Pilih Kualitas Kompresi DCT (0-100):", 0, 100, 50)

            if st.button('Kompresi'):
                if algorithm == "DWT":
                    coeffs = dwt_compress(audio, wavelet=wavelet, level=level)
                    compressed_audio = dwt_decompress(coeffs, wavelet=wavelet)
                elif algorithm == "DCT":
                    compressed_audio = dct_compress(audio, quality=quality)

                compressed_audio_bytes = write_audio(compressed_audio, params)
                st.audio(compressed_audio_bytes, format='audio/wav', start_time=0)

                st.download_button(
                    label="Unduh Audio Kompresi",
                    data=compressed_audio_bytes,
                    file_name="compressed_audio.wav",
                    mime="audio/wav"
                )
                st.success("Kompresi audio berhasil!")

if __name__ == "__main__":
    main()
